In [13]:
import requests

url = "https://api.apilayer.com/exchangerates_data/timeseries?start_date=2022-01-01&end_date=2022-09-30"

payload = {}
headers= {
  "apikey": ""
}

response = requests.request("GET", url, headers=headers, data = payload)

status_code = response.status_code
result = response.json()

In [97]:
from datetime import datetime,timedelta
import requests
import urllib.parse
from collections import defaultdict
import json


def days_between(d1, d2):
    '''
    Get difference between two datetimes in days
    '''
    return abs((d2 - d1).days)

def get_exchange_rates(start,end,base='USD', symbols='RUB,CAD', api_key = None):
    '''
    Get exchange rates for a given period, base currency, and relative currency symbols . 
    Uses `/timeseries` endpoint at https://apilayer.com/marketplace/exchangerates_data-api API which requires a key.
    Obtain api key here:
        https://exchangeratesapi.io/

    Args:
      - start,end : str - date string in the format `%Y-%m-%d`
      - base : str - base currency tricode
      - symbols : str - string of relative currency codes separated by a comma
      - api_key : str - API key

    Example:
        ```
        api_key = 'xxxx'
        data = get_exchange_rates('2013-01-01','2022-09-30', base = 'USD', symbols='RUB,CAD', api_key=api_key)
        print(data)

        # Output:
        {'2013-01-01': {'CAD': 0.993682, 'RUB': 30.503886, 'USD': 1.0},
        '2013-01-02': {'CAD': 0.987487, 'RUB': 30.250274, 'USD': 1.0},
        '2013-01-03': {'CAD': 0.987812, 'RUB': 30.251713, 'USD': 1.0},
        '2013-01-04': {'CAD': 0.986639, 'RUB': 30.278613, 'USD': 1.0},
        ```
    
    '''
    # make sure api key is present
    assert api_key is not None,'Requires api key. Obtain here https://exchangeratesapi.io/'
    
    # convert to datetime for calculations
    start = datetime.strptime(start, "%Y-%m-%d") 
    end = datetime.strptime(end, "%Y-%m-%d") 
    if days_between(start,end) > 365: # check if difference is above api threshild
        # break up date range
        ranges = [[start,start+timedelta(days=364)],[start+timedelta(days=365), end]]
        data = []
        for r in ranges: # for each date range get the data
            # using recursion keep breaking up the date range
            data.append(get_exchange_rates(start = r[0].strftime("%Y-%m-%d"),
                                           end = r[1].strftime("%Y-%m-%d") ,
                                           base = base, 
                                           symbols = symbols, 
                                           api_key = api_key))

        # merge list of dictionaries with data
        result = {k: v for d in data for k, v  in d.items()}
        return result


    try: # try getting the data
        # convert back to string
        start = start.strftime("%Y-%m-%d")
        end = end.strftime("%Y-%m-%d")
        # create request url
        url = f"https://api.apilayer.com/exchangerates_data/timeseries?start_date={start}&end_date={end}&base={base}&symbols={symbols}"
        # add api key to headers
        headers= {
            "apikey": api_key
            }
        # make request, get response
        response = requests.request("GET", url, headers=headers, data = payload)
        # get status code
        status_code = response.status_code
        # get exchange data
        result = response.json()['rates']
        # add base currency constant
        result = {k:{**{base:1.0},**v} for k,v in result.items()}
        return result
    except Exception as e: # print response if error
        print(response.json())
        

In [246]:
api_key = "208OOGLVF2wrGKnEn9RG2oj8JRaMv7C2"
data = get_exchange_rates('2013-01-01','2022-09-30', api_key=api_key)
data

{'2013-01-01': {'USD': 1.0, 'RUB': 30.503886, 'CAD': 0.993682},
 '2013-01-02': {'USD': 1.0, 'RUB': 30.250274, 'CAD': 0.987487},
 '2013-01-03': {'USD': 1.0, 'RUB': 30.251713, 'CAD': 0.987812},
 '2013-01-04': {'USD': 1.0, 'RUB': 30.278613, 'CAD': 0.986639},
 '2013-01-05': {'USD': 1.0, 'RUB': 30.307263, 'CAD': 0.98716},
 '2013-01-06': {'USD': 1.0, 'RUB': 30.306813, 'CAD': 0.98674},
 '2013-01-07': {'USD': 1.0, 'RUB': 30.313763, 'CAD': 0.986132},
 '2013-01-08': {'USD': 1.0, 'RUB': 30.313749, 'CAD': 0.986488},
 '2013-01-09': {'USD': 1.0, 'RUB': 30.368385, 'CAD': 0.987438},
 '2013-01-10': {'USD': 1.0, 'RUB': 30.249567, 'CAD': 0.984558},
 '2013-01-11': {'USD': 1.0, 'RUB': 30.28222, 'CAD': 0.984644},
 '2013-01-12': {'USD': 1.0, 'RUB': 30.274387, 'CAD': 0.984058},
 '2013-01-13': {'USD': 1.0, 'RUB': 30.27411, 'CAD': 0.983969},
 '2013-01-14': {'USD': 1.0, 'RUB': 30.201564, 'CAD': 0.984608},
 '2013-01-15': {'USD': 1.0, 'RUB': 30.273041, 'CAD': 0.984336},
 '2013-01-16': {'USD': 1.0, 'RUB': 30.329954

In [247]:
data = [{'date':k,'rate_rub':v.get('RUB'),'rate_cad':v.get('CAD')} for k,v in data.items()]

In [248]:
with open('exchange_rates.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [249]:
FRED_API_KEY = ''
# series_id = 'DCOILBRENTEU'
series_id = 'DCOILWTICO'
start = '2013-01-01'
end = '2022-09-30'
url = f'''https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json&observation_start={start}&observation_end={end}'''
print(url)

https://api.stlouisfed.org/fred/series/observations?series_id=DCOILWTICO&api_key=750b132fab29a57741b6e806b97cebfd&file_type=json&observation_start=2013-01-01&observation_end=2022-09-30


In [250]:


def get_fred_series(start,end,series_ids='DCOILBRENTEU,DCOILWTICO',api_key=None):
        '''
        Get FRED series observations for a given date range, and series ids.
        Uses `observations` endpoint at FRED API https://fred.stlouisfed.org/docs/api/fred/series_observations.html
        Obtain API key here:
                https://fred.stlouisfed.org/docs/api/api_key.html
        
        Args:
          - start,end : str - date string in the format `%Y-%m-%d`
          - series_ids : str - string of FRED series ids separated by a comma
          - api_key : str - API key
        
        Example:
          ```
          FRED_API_KEY = 'xxx'
          data = get_fred_series(start,end,'DCOILWTICO,DCOILBRENTEU',FRED_API_KEY)
          data

          # Output
          {'2022-01-03': {'DCOILWTICO': 75.99, 'DCOILBRENTEU': 78.25},
          '2022-01-04': {'DCOILWTICO': 77.0, 'DCOILBRENTEU': 79.39},
          '2022-01-05': {'DCOILWTICO': 77.83, 'DCOILBRENTEU': 80.6},
          ```
        '''
        # check for api_key
        assert api_key is not None, 'API key required. Obtain here https://fred.stlouisfed.org/docs/api/api_key.html'
        
        if len(series_ids.split(',')) > 1: # if more than 1 series get each using recursion
                series = defaultdict(dict)
                for s in series_ids.split(','): # merge series by key (date)
                        for key,value in get_fred_series(start,end,s,api_key).items():
                                series[key].update(value)
                return dict(series)
        
        # build url
        base_url = f'https://api.stlouisfed.org/fred/series/observations?file_type=json&'
        params = {'observation_start':start,
                'observation_end': end,
                'series_id': series_ids, 
                'api_key':FRED_API_KEY}
        url = base_url + urllib.parse.urlencode(params)

        try:
                # make request, get response
                response = requests.request("GET", url)
                # get status code
                status_code = response.status_code
                # get series data
                data = response.json()['observations']
                # convert to float, if value has digits
                result = {d['date']:{series_ids:float(d['value'])} for d in data if any(char.isdigit() for char in d['value'])}
                return result
        except Exception as e:
                print(response.json())
   


In [251]:
data = get_fred_series(start,end,'DCOILWTICO,DCOILBRENTEU',FRED_API_KEY)
data

{'2013-01-02': {'DCOILWTICO': 93.14, 'DCOILBRENTEU': 112.98},
 '2013-01-03': {'DCOILWTICO': 92.97, 'DCOILBRENTEU': 113.03},
 '2013-01-04': {'DCOILWTICO': 93.12, 'DCOILBRENTEU': 112.58},
 '2013-01-07': {'DCOILWTICO': 93.2, 'DCOILBRENTEU': 112.49},
 '2013-01-08': {'DCOILWTICO': 93.21, 'DCOILBRENTEU': 113.03},
 '2013-01-09': {'DCOILWTICO': 93.08, 'DCOILBRENTEU': 113.07},
 '2013-01-10': {'DCOILWTICO': 93.81, 'DCOILBRENTEU': 112.97},
 '2013-01-11': {'DCOILWTICO': 93.6, 'DCOILBRENTEU': 110.3},
 '2013-01-14': {'DCOILWTICO': 94.27, 'DCOILBRENTEU': 111.32},
 '2013-01-15': {'DCOILWTICO': 93.26, 'DCOILBRENTEU': 111.72},
 '2013-01-16': {'DCOILWTICO': 94.28, 'DCOILBRENTEU': 110.97},
 '2013-01-17': {'DCOILWTICO': 95.49, 'DCOILBRENTEU': 111.01},
 '2013-01-18': {'DCOILWTICO': 95.61, 'DCOILBRENTEU': 111.71},
 '2013-01-22': {'DCOILWTICO': 96.09, 'DCOILBRENTEU': 112.72},
 '2013-01-23': {'DCOILWTICO': 95.06, 'DCOILBRENTEU': 113.68},
 '2013-01-24': {'DCOILWTICO': 95.35, 'DCOILBRENTEU': 114.59},
 '2013-01-2

In [252]:
data = [{'date':k,'wti_price':v.get('DCOILWTICO'),'brent_price':v.get('DCOILBRENTEU')} for k,v in data.items()]

In [253]:
with open('oil_prices.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
